# Merge Files to Plain Text

A reusable, documented notebook for consolidating files from a folder into a single `.txt` file for downstream tasks (search, LLM context building, archiving, etc.).

## Overview

This notebook recursively scans a folder, filters by allowed file extensions, and merges file contents into a single plain-text output.
It is designed to be **copy/paste friendly** for open-source use and reproducible across environments.

### Features
- Simple, configurable **parameters** section (no code changes needed).
- Robust file handling with sensible defaults (UTF-8 with graceful fallbacks).
- Skips binary or oversized files (configurable), with a per-file header to retain provenance.
- Optional exclusion patterns (e.g., `node_modules`, `.git`, build artifacts).
- Deterministic output ordering for reproducibility.

### What it does *not* do (by default)
- Parse proprietary binary formats (e.g., `.docx`, `.pdf`). You can add readers in the `READERS` map if desired.
- Preserve exact original encodings—content is normalized to UTF-8 in the merged file.


## Requirements

- Python 3.8+
- Standard library only (no external dependencies)


## Quick Start

1. Set the parameters in the **Configuration** cell below (at minimum `INPUT_DIR` and `OUTPUT_TXT`).  
2. Run the notebook cells from top to bottom.  
3. Find the merged text at the path you configured in `OUTPUT_TXT`.


In [1]:
# === Configuration ===
from pathlib import Path

# Folder to scan (recursive). Use an absolute or relative path.
INPUT_DIR = Path(r"C:\Users\keith\Downloads\resume-pdf-layouter")  # e.g., Path("/path/to/project")

# Where to write the merged text file.
OUTPUT_TXT = Path(r"C:\Users\keith\Downloads\resume-pdf-layouter-altogether.txt")

# Include files with these extensions (lowercase, include the dot).
INCLUDE_EXTS = {
    ".txt", ".md", ".py", ".json", ".yaml", ".yml", ".toml", ".tsx", ".mjs",
    ".csv", ".tsv", ".xml", ".ini", ".cfg", ".html", ".css", ".js", ".ts"
}

# Exclude paths that **contain** any of these substrings (case-insensitive).
EXCLUDE_SUBSTRINGS = {"node_modules", ".git", "__pycache__", "build", "dist", ".venv", ".mypy_cache", 
                      "resume_pdf_layouter_mvp_full_repo", "resume-pdf-layouter-altogether.txt", "package-lock",
                      ".next", "sample-resume.md"}

# Max size per file in bytes (skip if larger). Set to None to disable.
MAX_FILE_BYTES = None # 2 * 1024 * 1024  # 2 MB

# Whether to include a header with file path and size before each file's content.
INCLUDE_FILE_HEADERS = True

# Whether to include an end-of-file marker after each file.
INCLUDE_EOF_MARKERS = True

# If True, use relative paths to INPUT_DIR in headers; otherwise use absolute paths.
USE_RELATIVE_PATHS = True


In [2]:
# === Helpers ===
from pathlib import Path
import os

def is_excluded(path: Path, exclude_substrings):
    p = str(path).lower()
    return any(substr.lower() in p for substr in exclude_substrings)

def safe_read_text(path: Path, max_bytes=None) -> str:
    """
    Read a file as text using UTF-8 with fallbacks. Truncates if max_bytes is specified.
    Returns the (possibly truncated) text content.
    """
    if max_bytes is not None and path.stat().st_size > max_bytes:
        raise ValueError(f"File too large ({path.stat().st_size} bytes)")

    # Basic binary sniff: read a small chunk to detect null bytes
    try:
        with open(path, "rb") as rb:
            head = rb.read(2048)
            if b"\x00" in head:
                raise UnicodeDecodeError("utf-8", head, 0, 1, "binary data detected")
    except Exception as e:
        # If we fail to open in binary for sniffing, we'll try text anyway
        pass

    # Try utf-8 first, then latin-1 as a last resort to avoid crashing
    try:
        return path.read_text(encoding="utf-8", errors="strict")
    except Exception:
        try:
            return path.read_text(encoding="utf-8", errors="replace")
        except Exception:
            return path.read_text(encoding="latin-1", errors="replace")

def discover_files(root: Path, include_exts, exclude_substrings):
    files = []
    for p in sorted(root.rglob("*")):
        if not p.is_file():
            continue
        if include_exts and p.suffix.lower() not in include_exts:
            continue
        if is_excluded(p, exclude_substrings):
            continue
        files.append(p)
    return files

def format_header(path: Path, root: Path, size: int) -> str:
    shown = path.relative_to(root) if USE_RELATIVE_PATHS else path.resolve()
    return f"""
===== FILE: {shown} | SIZE: {size} bytes =====
"""

def format_footer() -> str:
    return "\n===== END FILE =====\n"


In [3]:
# === Merge Execution ===
from datetime import datetime

root = INPUT_DIR.resolve()
files = discover_files(root, INCLUDE_EXTS, EXCLUDE_SUBSTRINGS)

if not files:
    print("No files found with current filters.")
else:
    print(f"Discovered {len(files)} files. Merging...")

written = 0
skipped = 0
err_files = []
with open(OUTPUT_TXT, "w", encoding="utf-8") as out:
    # Provenance header
    out.write("# Merged Text Export\n")
    out.write(f"# Source root: {root}\n")
    out.write(f"# Timestamp: {datetime.utcnow().isoformat()}Z\n")
    out.write(f"# Files included: {len(files)}\n\n")

    for f in files:
        try:
            content = safe_read_text(f, max_bytes=MAX_FILE_BYTES)
            if INCLUDE_FILE_HEADERS:
                out.write(format_header(f, root, f.stat().st_size))
            out.write(content)
            if INCLUDE_EOF_MARKERS:
                out.write(format_footer())
            written += 1
        except Exception as e:
            skipped += 1
            err_files.append((str(f), str(e)))

print(f"Done. Wrote {written} files into: {OUTPUT_TXT}")
if skipped:
    print(f"Skipped {skipped} files due to errors or filters.")
if err_files:
    print("First few errors:")
    for item in err_files[:5]:
        print(" -", item[0], "->", item[1])


Discovered 23 files. Merging...


C:\Users\keith\AppData\Local\Temp\ipykernel_10052\1901250185.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  out.write(f"# Timestamp: {datetime.utcnow().isoformat()}Z\n")


Done. Wrote 23 files into: C:\Users\keith\Downloads\resume-pdf-layouter-altogether.txt


In [4]:
# === Validate Output & Preview ===
from pathlib import Path

if Path(OUTPUT_TXT).exists():
    print("Output size:", Path(OUTPUT_TXT).stat().st_size, "bytes")
    # Show the first ~1000 chars for a quick check (non-destructive)
    preview = Path(OUTPUT_TXT).read_text(encoding="utf-8", errors="replace")[:1000]
    print(preview)
else:
    print("No output found. Did the merge run successfully?")


Output size: 47556 bytes
# Merged Text Export
# Source root: C:\Users\keith\Downloads\resume-pdf-layouter
# Timestamp: 2025-09-11T02:33:53.007676Z
# Files included: 23


===== FILE: app\api\render\route.ts | SIZE: 1190 bytes =====
// app/api/render/route.ts
// File: app/api/render/route.ts
import { NextRequest } from "next/server";
import { z } from "zod";
import { mdToHtml } from "@/lib/markdown";
import { renderBestPdf, renderSinglePdf } from "@/lib/optimizer";
import { Layout, PageSize } from "@/lib/types";

const Body = z.object({
  markdown: z.string(),
  layout: z.enum(["one", "two-eq", "two-4060"]).default("one"),
  pageSize: z.enum(["letter", "a4"]).default("letter"),
  optimize: z.boolean().default(true),
  maxPages: z.number().int().min(1).max(10).optional().nullable(), // NEW
});

export async function POST(req: NextRequest) {
  const json = await req.json();
  const body = Body.parse(json);
  const rendered = mdToHtml(body.markdown);

  const pdf = body.optimize
    ? await

## Extending This Notebook

- To support other file types (e.g., `.pdf`, `.docx`), add specialized readers in a `READERS` map and update `discover_files` to include those extensions.  
- To add a CLI, convert the logic into a standalone script and parse args with `argparse`.  
- For large repositories, consider stream processing or chunking the output by directory.


## License

This notebook is provided under the MIT License. See `LICENSE` in your repository for details.


In [5]:
# === Environment Info (Optional) ===
import sys, platform
print("Python:", sys.version.replace("\n"," "))
print("Platform:", platform.platform())


Python: 3.13.2 (tags/v3.13.2:4f8bb39, Feb  4 2025, 15:23:48) [MSC v.1942 64 bit (AMD64)]
Platform: Windows-11-10.0.26100-SP0
